In [10]:
from yahoofinancials import YahooFinancials
import numpy as np
import wbdata
import pandas as pd
import math
from datetime import datetime
import requests
import json
import random
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import csv

# tickers = ['AAPL','GE','AMZN','KO','SBUX','OXY','F','AMD','MSFT','T']
# tickers = ['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AGN', 'AIG', 'ALL', 'AMGN', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BKNG', 'BLK', 'BMY', 'BRK-B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DUK', 'EMR', 'EXC', 'F', 'FB', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTN', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'UTX', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM']
# del_tickers = ['DOW']
tickers = ['AAPL', 'ABT', 'ACN', 'ADBE', 'AGN', 'AIG', 'ALL', 'AMGN', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BKNG', 'BLK', 'BMY', 'BRKB', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DUK', 'EMR', 'EXC', 'F', 'FDX', 'GD', 'GE', 'GILD', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'QCOM', 'RTN', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'UTX', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM']

# indicator_income = ['Revenue', 'Gross Profit', 'R&D Expenses', 'Operating Income', 'Operating Expenses',\
#                  'Interest Expense', 'Net Income','EBIT','EBITDA']
indicator_income = ['revenue','grossProfit','researchAndDevelopmentExpenses','operatingExpenses','interestExpense','depreciationAndAmortization',\
                   'ebitda','operatingIncome','incomeBeforeTax','netIncome']

fsa_dict = {}
for x in indicator_income:
    fsa_dict[x] = []
    
# indicator_balance = ['Cash and cash equivalents','Short-term investments','Receivables','Inventories','Goodwill and Intangible Assets',\
#                    'Total assets','Total liabilities','Total current assets','Total current liabilities','Total shareholders equity']
indicator_balance = ['cashAndCashEquivalents','shortTermInvestments','netReceivables','inventory','totalCurrentAssets','goodwill','intangibleAssets',\
                   'shortTermDebt','totalCurrentLiabilities','totalLiabilities','retainedEarnings','totalStockholdersEquity','totalDebt','totalAssets']
for x in indicator_blance:
    fsa_dict[x] = []

indicator = indicator_income + indicator_blance
ratio = {}

    
def add_month(ts, m):
    ts_adj = datetime(ts.year + (ts.month + m // 12), ((ts.month + m % 12)), 1).strftime("%Y-%m-%d")
    return ts_adj

def to_txt(data, name,ticker):
    temp = 'a+'
    if ticker == 'AAPL':temp = 'w'
    with open(name, temp) as f:
        for i2, x in enumerate(data[:-1]):
            for i in range(1,len(x)):
                f.write(str(x[i]))
                f.write(',')
                if i == len(x)-1:
                    f.write(str(data[i2+1][1]))
                    
            f.write('\n')

# 财报数据(Ratio) 
参考Bankruptcy prediction using imaged financial ratios and convolutional neural networks

In [20]:
def get_financialstatement_ratio(ticker,indicator,indicator_income,indicator_balance,start_date,end_date):
    stock_dict = {}
    for x in indicator:
        stock_dict[x] = []

    # income_statement
    X = []
    with open('income-statement/%s.csv'%ticker,newline='') as f:
        rows = list(csv.reader(f))
        column = rows[0]
        for row in rows[1:]:
            if start_date <= row[0] <= end_date: # row[0] == date
                X.append(row)
    
    X = sorted(X)  # 照日期排序

    for _ in range(4-len(X)): # 未必有4个季度的资料，为了方便处理补0
        for i,x in enumerate(indicator_income):
            stock_dict[x].append(0)

    for a in range(len(X)):
        for x in indicator_income:
            stock_dict[x].append(float(X[a][column.index(x)]))
            
    # balance sheet
    X = []
    with open('balance-sheet-statement/%s.csv'%ticker,newline='') as f:
        rows = list(csv.reader(f))
        column = rows[0]
        for row in rows[1:]:
#             print(row)
            if start_date <= row[0] <= end_date: # row[0] == date
                X.append(row)
    
    X = sorted(X)  # 照日期排序

    for _ in range(4-len(X)): # 未必有4个季度的资料，为了方便处理补0
        for i,x in enumerate(indicator_balance):
            stock_dict[x].append(0)

    for a in range(len(X)):
        for x in indicator_balance:
            stock_dict[x].append(float(X[a][column.index(x)]))
    
    return stock_dict
get_financialstatement_ratio('AAPL',indicator,indicator_income,indicator_balance,'2009-05-01','2010-05-01')

{'revenue': [8337000000.0, 9870000000.0, 15683000000.0, 29182000000.0],
 'grossProfit': [3023000000.0, 3614000000.0, 6411000000.0, 12036000000.0],
 'researchAndDevelopmentExpenses': [341000000.0,
  358000000.0,
  398000000.0,
  824000000.0],
 'operatingExpenses': [1351000000.0, 1421000000.0, 1686000000.0, 3332000000.0],
 'interestExpense': [0.0, 0.0, 0.0, 0.0],
 'depreciationAndAmortization': [176000000.0,
  197000000.0,
  209000000.0,
  425000000.0],
 'ebitda': [1848000000.0, 2390000000.0, 4934000000.0, 9129000000.0],
 'operatingIncome': [1672000000.0, 2193000000.0, 4725000000.0, 8704000000.0],
 'incomeBeforeTax': [1732000000.0, 2238000000.0, 4758000000.0, 8787000000.0],
 'netIncome': [1229000000.0, 1665000000.0, 3378000000.0, 6452000000.0],
 'cashAndCashEquivalents': [5605000000.0,
  5263000000.0,
  7609000000.0,
  10018000000.0],
 'shortTermInvestments': [18617000000.0,
  18201000000.0,
  17187000000.0,
  13137000000.0],
 'netReceivables': [2686000000.0, 3361000000.0, 3090000000.0, 

# Object Function

In [13]:
def object_function(pic,corr,ratio_adj):
    num_set = set()
    for _ in range(3*(len(ratio_adj))):
        duplicate = 1
        while duplicate:
            num1 = random.randint(0,len(ratio_adj)-1)
            num2 = random.randint(0,len(ratio_adj)-1)
            if num1 > num2:
                num1, num2 = num2, num1
            if num1 != num2 and (num1,num2) not in num_set:
                num_set.add((num1,num2))
                duplicate = 0
        
    for x,y in list(num_set):
        old = 0
        for num in range(len(ratio_adj)):
            distance = (num//11 - x//11)**2 + (num%11 - x%11)**2
            cor = min(corr.get(pic[x]+'_'+pic[num],1),corr.get(pic[num]+'_'+pic[x],1))
            old += cor*distance
        for num in range(len(ratio_adj)):
            distance = (num//11 - y//11)**2 + (num%11 - y%11)**2
            cor = min(corr.get(pic[y]+'_'+pic[num],1),corr.get(pic[num]+'_'+pic[y],1))
            old += cor*distance
            
        for num in range(len(ratio_adj)):
            distance = (num//11 - x//11)**2 + (num%11 - x%11)**2
            cor = min(corr.get(pic[y]+'_'+pic[num],1),corr.get(pic[num]+'_'+pic[y],1))
            old -= cor*distance
        for num in range(len(ratio_adj)):
            distance = (num//11 - y//11)**2 + (num%11 - y%11)**2
            cor = min(corr.get(pic[x]+'_'+pic[num],1),corr.get(pic[num]+'_'+pic[x],1))
            old -= cor*distance
            
        if old > 0:
            pic[x], pic[y] = pic[y], pic[x]
            return True
    return False

def energy(pic,corr,ratio_adj):
    result = 0
    for x in range(len(ratio_adj)):
        for y in range(x+1,len(ratio_adj)):
            distance = (y//11 - x//11)**2 + (y%11 - x%11)**2
            cor = min(corr.get(pic[x]+'_'+pic[y],1),corr.get(pic[y]+'_'+pic[x],1))
            result += cor*distance
    return int(result)

def optimize_pixels():
    # del ratio with too much missing value
    ratio_adj = []
    for x,y in ratio.items():
        if y.count(float('inf')) <= len(y)*0.3:
            ratio_adj.append([x,y])

    pic = [] 
    corr = {}

    # for _ in range(11):
    #     pic.append([127]*11)
    # for i,x in enumerate(ratio_adj):
    #     pic[i//11][i%11] = x[0]
    for i,x in enumerate(ratio_adj):
        pic.append(x[0])

    # 预先算好corr
    for i,x in enumerate(ratio_adj[-3:]):
        for y in ratio_adj[i+1:]:
            miss_index = set() # miss value的index
            for i2,x2 in enumerate(x[1]):
                if x2 == float('inf'):miss_index.add(i2)
            for i2,x2 in enumerate(y[1]):
                if x2 == float('inf'):miss_index.add(i2)

            list_x = [x2 for i2,x2 in enumerate(x[1]) if i2 not in miss_index]
            list_y = [x2 for i2,x2 in enumerate(y[1]) if i2 not in miss_index]
            corr[x[0]+'_' + y[0]] = abs(stats.pearsonr(list_x,list_y)[0])

    cnt = 0
    random.shuffle(pic)
    print(energy(pic,corr,ratio_adj))

    
    while object_function(pic,corr,ratio_adj): 
        cnt += 1
#         if cnt%100 == 0:
#             print(cnt,energy(pic,corr,ratio_adj))
    print(cnt,energy(pic,corr,ratio_adj))
    return pic,ratio_adj,energy(pic,corr,ratio_adj)


In [24]:
def generate_corr_pic():
    fsa_dict = {}
    for x in indicator:
        fsa_dict[x] = []
    
    for ticker in tickers:
        stock_dict = get_financialstatement_ratio(ticker,indicator,indicator_income,indicator_balance,'2009-05-10','2010-05-10')
        for x in indicator:
            fsa_dict[x] += stock_dict[x]
        print(ticker,len(fsa_dict['revenue']))

    for i,x in enumerate(indicator):
        for y in indicator[i+1:]:
            if x != y:
                for z in range(len(fsa_dict[x])):
                    fsa_x = [num for num in fsa_dict[x] if num != 0 and num != float('inf')]
                    fsa_y = [num for num in fsa_dict[y] if num != 0 and num != float('inf')]
                    avg_x = sum(fsa_x)/len(fsa_x)
                    avg_y = sum(fsa_y)/len(fsa_y)
                    if avg_y < avg_x:
                        indicator_x = y
                        indicator_y = x
                    else:
                        indicator_x = x
                        indicator_y = y
                    ratio[indicator_x+'/'+indicator_y] = ratio.get(indicator_x+'/'+indicator_y,[])
                    if fsa_dict[indicator_x][z] == float('inf') or fsa_dict[indicator_y][z] == float('inf') or fsa_dict[indicator_y][z] == 0:
                        ratio[indicator_x+'/'+indicator_y].append(float('inf'))
                    else:
                        ratio[indicator_x+'/'+indicator_y].append(fsa_dict[indicator_x][z]/fsa_dict[indicator_y][z])
    # count the pixels
    p = 0
    e = float('inf')
    for _ in range(10):
        print(_)
        pic,ratio_adj,e1 = optimize_pixels()
        if e1 < e:
            e = e1
            p = pic

    with open('pic.txt', 'w') as f:
        for x in pic:
            f.write(x)
            f.write(',')
generate_corr_pic()

AAPL 4
ABT 8
ACN 12
ADBE 16
AGN 20
AIG 24
ALL 28
AMGN 32
AMZN 36
AXP 40
BA 44
BAC 48
BIIB 52
BK 56
BKNG 60
BLK 64
BMY 68
BRKB 72
C 76
CAT 80
CHTR 84
CL 88
CMCSA 92
COF 96
COP 100
COST 105
CSCO 109
CVS 113
CVX 117
DD 121
DHR 125
DIS 129
DUK 133
EMR 137
EXC 141
F 145
FDX 149
GD 153
GE 157
GILD 161
GOOG 165
GOOGL 169
GS 173
HD 177
HON 181
IBM 185
INTC 189
JNJ 193
JPM 197
KO 201
LLY 205
LMT 209
LOW 213
MA 217
MCD 221
MDLZ 225
MDT 229
MET 233
MMM 237
MO 241
MRK 245
MS 249
MSFT 253
NEE 257
NFLX 261
NKE 265
NVDA 269
ORCL 273
OXY 277
PEP 281
PFE 285
PG 289
PM 293
QCOM 297
RTN 301
SBUX 305
SLB 309
SO 313
SPG 317
T 321
TGT 325
TMO 329
TXN 333
UNH 337
UNP 341
UPS 345
USB 349
UTX 353
V 357
VZ 361
WBA 365
WFC 369
WMT 373
XOM 377
0


ValueError: x and y must have the same length.

In [5]:
def get_financialstatement_ratio_graph(ticker,indicator,link,start_date,end_date):
    ticker = ticker.replace('-','.')
    # income statement
    X = []
    for x in fs:
        if start_date <= x['date'] <= end_date:
            
            temp = [x['date']]
            for i in indicator:
                if x[i] == "":
                    temp.append(float('inf'))
                else:
                    temp.append(float(x[i]))
            X.append(temp)
    X = sorted(X)  # 照日期排序
    for a in range(len(X)):
        for i,x in enumerate(indicator):
            if a == 0:
                fsa_ticker_dict[x][ticker] = [[X[a][0],X[a][i+1]]]
            else:
                fsa_ticker_dict[x][ticker].append([X[a][0],X[a][i+1]])
    
    return X

In [6]:
def generate_fsa_data():
    indicator_income = ['Revenue', 'Gross Profit', 'R&D Expenses', 'Operating Income', 'Operating Expenses',\
                     'Interest Expense', 'Net Income','EBIT','EBITDA']
    income_link = "https://financialmodelingprep.com/api/v3/financials/income-statement/%s?period=quarter"
    fsa_ticker_dict = {}
    for x in indicator_income:
        fsa_ticker_dict[x] = {}

    indicator_blance = ['Cash and cash equivalents','Short-term investments','Receivables','Inventories','Goodwill and Intangible Assets',\
                       'Total assets','Total liabilities','Total current assets','Total current liabilities','Total shareholders equity']
    blance_link = "https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/%s?period=quarter"
    for x in indicator_blance:
        fsa_ticker_dict[x] = {}

    indicator = indicator_income + indicator_blance
    ratio = {}

    for ticker in tickers:
        print(ticker)
        # 季报多数在12月结算，也有11 or 1月
        get_financialstatement_ratio_graph(ticker,indicator_income,income_link,'2011-02-01','2019-11-31')  
        get_financialstatement_ratio_graph(ticker,indicator_blance,blance_link,'2011-02-01','2019-11-31')


    # 整理财报数据 格式: 指标 代码 数据1 数据2 ...
    start_quarter = datetime.strptime("2011-02-15", "%Y-%m-%d")
    quarter = []
    head = ['indicator','ticker']
    for x in range(35):
        head.append(datetime.strftime(start_quarter, "%Y") + 'Q' + str(x%4+1))
        quarter.append(start_quarter)
        start_quarter += relativedelta(months=3)

    text = []

    for i in indicator:
        for t in tickers:
            t = t.replace('-','.')
            temp = [i,t]
            for q in quarter:
                lower = datetime.strftime(q, "%Y-%m-%d")
                upper = datetime.strftime(q+relativedelta(months=3), "%Y-%m-%d")
                for data in fsa_ticker_dict[i][t]:
                    if data[0] >= lower:
                        temp.append(data[1])
                        break
            text.append(temp)

    # write into txt
    with open('fsa.txt', 'w') as f:
        for x in [head] + text:
            for y in x:
                word = str(y)
                f.write(word)
                f.write(',')
            f.write('\n')

# 建立财报灰度

In [23]:
import os
def brightness():
    abs_file_path = os.path.join(os.path.dirname('__file__'), 'fsa.txt')
    with open(abs_file_path,'r') as f:
        lines = f.read()
    data = lines.split('\n')
    head = data[0].split(',')[:-1]
    data.pop(0)

    # 读取之前存好的财报数据，转成 指标:季度:[...] 格式
    raw_dict = {}  
    for i in indicator:
        raw_dict[i] = {}
        for x in head[2:]:
            raw_dict[i][x] = []
    for d in data[:-1]:
        num = d.split(',')
        for x in range(2,len(head)):
            raw_dict[num[0]][head[x]].append(float(num[x]))
            
    # 根据pic.txt的要求计算ratio
    with open('pic.txt','r') as f:
        lines = f.read()
    data = lines.split(',')[:-1]
    
    ratio_dict = {}  
    for ratio in data:
        ratio_dict[ratio] = {}
        ind1, ind2 = ratio.split('/')
        for quarter in head[2:]:
            temp_ratio = []
            #ratio_dict[ratio][quarter] = []
            for x1,x2 in zip(raw_dict[ind1][quarter],raw_dict[ind2][quarter]):
                if x1 == float('inf') or x2 == float('inf') or x2 == 0:
                    temp_ratio.append(float('inf'))
                else:
                    temp_ratio.append(x1/x2)
            
            # ratio to gray color
            std = np.std([num for num in temp_ratio if num != float('inf')])
            avg = np.average([num for num in temp_ratio if num != float('inf')])
            ratio_dict[ratio][quarter] = []
            for num in temp_ratio:
                if num == float('inf'):
                    ratio_dict[ratio][quarter].append(128)
                else:
                    ratio_dict[ratio][quarter].append(min(max((num-avg)/std*100+128,0),255))
    
    plt.figure(figsize=(18,18))
    cnt = 0
    pic_result = []
    for i,ticker in enumerate(tickers):
        for quarter in head[2:]:
            grayscale_pic = []
            for ratio in data:
                grayscale_pic.append(ratio_dict[ratio][quarter][i])
            while len(grayscale_pic) < 121:
                grayscale_pic.append(128)
            grayscale_pic = np.reshape(grayscale_pic,(11,11))
#             plt.subplot(7, 5, cnt + 1)
#             cnt += 1
#             plt.imshow(grayscale_pic, cmap="gray")
            pic_result.append([ticker,quarter,grayscale_pic])
    return pic_result


In [24]:
if __name__ == '__main__':
    pic = brightness()
    print(len(pic))
    print(pic[2])

3290
['AAPL', '2011Q3', array([[102.67635963, 107.37214172,  80.13200843,  65.35292919,
         99.00625412,  70.51972244,  82.68613229,  96.31097722,
        129.21093554, 119.22835127,  88.63563713],
       [107.56542546,  96.71795451,  65.02359011, 134.11963684,
        189.92502437, 172.54276158, 114.0457553 , 158.75449371,
        128.84467061,  91.83243719,  97.53569758],
       [164.3007826 , 134.43749748, 116.88521401, 255.        ,
        250.07674268, 111.90318145, 177.48327471, 120.0391741 ,
        106.0363566 , 140.43545334, 116.24839946],
       [163.71991039, 163.18007601,   0.        , 129.15149164,
        114.4629847 , 181.15795861, 112.41311041, 147.58747417,
         86.9199687 , 148.68418455, 156.057222  ],
       [119.78192407, 130.43055227, 215.63059063, 255.        ,
        255.        , 105.97063086,  70.65555809, 119.18836678,
         67.78932874, 166.16778818, 167.37917356],
       [119.68955871,  71.69821412, 147.56097336, 185.90237554,
         47.34082

<Figure size 1296x1296 with 0 Axes>